In [28]:
!pip install pandas

  Using cached https://files.pythonhosted.org/packages/f1/e1/56bf2e1b92ce1361169aff7ba0316d211339cc8e2737f5f2e7a5930c9574/pandas-1.1.0-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/c7/7d/ea9e28c3a99f50e77ee9a0e3759adb6537b2bb7a84aef27b8c0ddc431b48/numpy-1.19.1-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/4f/a4/879454d49688e2fad93e59d7d4efda580b783c745fd2ec2a3adf87b0808d/pytz-2020.1-py2.py3-none-any.whl

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [30]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [2]:
base_url = "https://stackoverflow.com/tags/"
tag = "python"
query_filter = "Votes"
url = f"{base_url}{tag}?tab={query_filter}"
print(url)

https://stackoverflow.com/tags/python?tab=Votes


In [3]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

In [4]:
question_summaries = html.find('.question-summary')
print(question_summaries)

[<Element 'div' class=('question-summary',) id='question-summary-231767'>, <Element 'div' class=('question-summary',) id='question-summary-419163'>, <Element 'div' class=('question-summary',) id='question-summary-394809'>, <Element 'div' class=('question-summary',) id='question-summary-100003'>, <Element 'div' class=('question-summary',) id='question-summary-82831'>, <Element 'div' class=('question-summary',) id='question-summary-89228'>, <Element 'div' class=('question-summary',) id='question-summary-38987'>, <Element 'div' class=('question-summary',) id='question-summary-273192'>, <Element 'div' class=('question-summary',) id='question-summary-522563'>, <Element 'div' class=('question-summary',) id='question-summary-136097'>, <Element 'div' class=('question-summary',) id='question-summary-3437059'>, <Element 'div' class=('question-summary',) id='question-summary-952914'>, <Element 'div' class=('question-summary',) id='question-summary-3207219'>, <Element 'div' class=('question-summar

In [5]:
print(question_summaries[0].text)

10398
votes
41answers
2.3m views
What does the “yield” keyword do?
What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....
python iterator generator yield coroutine
asked Oct 23 '08 at 22:21
Alex. S.
118k1515 gold badges4949 silver badges6161 bronze badges


In [6]:
columns = ['votes','vote_title','num_answers','views','question','short_desc','tags','date','user','user_details']

In [7]:
# this_row = list(question_summaries[0].text.split('\n'))
# this_row

In [8]:
# len(this_row) == len(columns)

In [9]:
# row_data = dict(zip(columns,this_row))
# row_data

In [10]:
# for col_val,row_val in zip(columns,this_row):
#     print(col_val,row_val)

In [11]:
keynames = ['question','votes','tags']
classes_needed = ['.question-hyperlink','.vote','.tags']
this_question_element = question_summaries[0]
this_question_element.find('.question-hyperlink',first=True).text

'What does the “yield” keyword do?'

In [12]:
this_question_element.find('.vote',first=True).text.replace('\nvotes','')

'10398'

In [13]:
def clean_scrapped_data(text,keyname=None):
    if keyname == 'votes':
        return text.replace('\nvotes','')
    return text

In [14]:
datas = []
for q_el in question_summaries:
    question_data ={}
    for i, _class in enumerate(classes_needed):
        keyname = keynames[i]
        question_data[keyname] = clean_scrapped_data(q_el.find(_class,first=True).text,keyname=keyname)
        datas.append(question_data)
        
# datas[0]

In [15]:
def parse_tagged_page(html):
    question_summaries = html.find('.question-summary')
    keynames = ['question','votes','tags']
    classes_needed = ['.question-hyperlink','.vote','.tags']
    datas = []
    for q_el in question_summaries:
        question_data ={}
        for i, _class in enumerate(classes_needed):
            keyname = keynames[i]
            question_data[keyname] = clean_scrapped_data(q_el.find(_class,first=True).text,keyname=keyname)
            datas.append(question_data)
    return datas

In [17]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200,299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [56]:
def scrape_tag(tag="machine-learning",query_filter="Votes",max_pages=50,pagesize=25):
    base_url = "https://stackoverflow.com/tags/"
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        #print(url)
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [57]:
datas = scrape_tag()
#https://stackoverflow.com/questions/tagged/machine-learning?tab=votes&page=2&pagesize=15

In [60]:
len(datas)

7500

In [61]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,What is the role of the bias in neural networks?,825,machine-learning neural-network artificial-int...
1,What is the role of the bias in neural networks?,825,machine-learning neural-network artificial-int...
2,What is the role of the bias in neural networks?,825,machine-learning neural-network artificial-int...
3,What is the difference between a generative an...,595,algorithm machine-learning difference
4,What is the difference between a generative an...,595,algorithm machine-learning difference


In [62]:
df.shape

(7500, 3)

In [63]:
df.to_csv("python.csv",index=False)

In [34]:
# auto-tagging question bot